In [108]:
import pandas as pd
import io
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
print ('Ran')

Ran


In [109]:
app_train = pd.read_csv('D:/Project_Data/application_train.csv')
app_test = pd.read_csv('D:/Project_Data/application_test.csv')

print("Shape of training data: {}".format(app_train.shape))
print("Shape of test data: {}".format(app_test.shape))

Shape of training data: (307511, 122)
Shape of test data: (48744, 121)


In [110]:
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [111]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le_count = 0

In [112]:
# Iterate through the columns
for col in app_train:
    if app_train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(app_train[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train[col])
            # Transform both training and testing data
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1

In [113]:
print('%d columns were label encoded.' % le_count)

3 columns were label encoded.


In [114]:
app_train.select_dtypes(include=['object']).apply(pd.Series.nunique, axis=0)

CODE_GENDER                    3
NAME_TYPE_SUITE                7
NAME_INCOME_TYPE               8
NAME_EDUCATION_TYPE            5
NAME_FAMILY_STATUS             6
NAME_HOUSING_TYPE              6
OCCUPATION_TYPE               18
WEEKDAY_APPR_PROCESS_START     7
ORGANIZATION_TYPE             58
FONDKAPREMONT_MODE             4
HOUSETYPE_MODE                 3
WALLSMATERIAL_MODE             7
EMERGENCYSTATE_MODE            2
dtype: int64

In [115]:
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

In [116]:
print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

Training Features shape:  (307511, 243)
Testing Features shape:  (48744, 239)


In [117]:
train_labels = app_train['TARGET']
train_sk_id_curr = app_train['SK_ID_CURR']
test_sk_id_curr = app_test['SK_ID_CURR']

app_train.drop('SK_ID_CURR', inplace=True, axis=1)
app_test.drop('SK_ID_CURR', inplace=True, axis=1)

app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)
print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

Training Features shape:  (307511, 238)
Testing Features shape:  (48744, 238)


In [118]:
from sklearn.preprocessing import Imputer
imputer = Imputer(strategy = 'median')
imputer.fit(app_train)
app_train = imputer.transform(app_train)
app_test = imputer.transform(app_test)

In [119]:
from sklearn.model_selection import train_test_split, cross_val_score
import xgboost as xgb

In [120]:
ratio = (train_labels == 0).sum()/ (train_labels == 1).sum()
ratio

11.387150050352467

0 - Non-defaulter and 1- Defaulter

In [121]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import roc_auc_score

In [122]:
X_train, X_test, y_train, y_test = train_test_split(app_train, train_labels, test_size=0.2, stratify=train_labels, random_state=1)
print("Postive examples in train set: {}".format(np.sum(y_train==0)))
print("Negative examples in train set: {}".format(np.sum(y_train==1)))

print("Postive examples in test set: {}".format(np.sum(y_test==0)))
print("Negative examples in test set: {}".format(np.sum(y_test==1)))

Postive examples in train set: 226148
Negative examples in train set: 19860
Postive examples in test set: 56538
Negative examples in test set: 4965


In [123]:
clf = XGBClassifier(n_estimators=1000, objective='binary:logistic', gamma=0.1, subsample=0.5, scale_pos_weight=ratio )
clf.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric='auc', early_stopping_rounds=10)

[0]	validation_0-auc:0.684694
Will train until validation_0-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.69767
[2]	validation_0-auc:0.701373
[3]	validation_0-auc:0.705643
[4]	validation_0-auc:0.70868
[5]	validation_0-auc:0.710959
[6]	validation_0-auc:0.712839
[7]	validation_0-auc:0.714636
[8]	validation_0-auc:0.715176
[9]	validation_0-auc:0.717006
[10]	validation_0-auc:0.718343
[11]	validation_0-auc:0.720083
[12]	validation_0-auc:0.722222
[13]	validation_0-auc:0.723856
[14]	validation_0-auc:0.724811
[15]	validation_0-auc:0.725467
[16]	validation_0-auc:0.72659
[17]	validation_0-auc:0.727612
[18]	validation_0-auc:0.729942
[19]	validation_0-auc:0.73148
[20]	validation_0-auc:0.732952
[21]	validation_0-auc:0.73428
[22]	validation_0-auc:0.734449
[23]	validation_0-auc:0.734913
[24]	validation_0-auc:0.735365
[25]	validation_0-auc:0.735641
[26]	validation_0-auc:0.736876
[27]	validation_0-auc:0.737942
[28]	validation_0-auc:0.738443
[29]	validation_0-auc:0.738795
[30]	validation_0-auc

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0.1,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=1000, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=11.387150050352467,
       seed=None, silent=None, subsample=0.5, verbosity=1)

In [124]:
from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_test)
print('Accuracy score ',accuracy_score(y_test, y_pred))

Accuracy score  0.6991041087426629


In [125]:
n_estimators = clf.best_ntree_limit
clf = XGBClassifier(n_estimators=n_estimators, objective='binary:logistic', gamma=0.1, subsample=0.5, scale_pos_weight=ratio )
clf.fit(app_train, train_labels, eval_set=[(app_train, train_labels)], eval_metric='auc')

[0]	validation_0-auc:0.686937
[1]	validation_0-auc:0.691054
[2]	validation_0-auc:0.706216
[3]	validation_0-auc:0.708388
[4]	validation_0-auc:0.708699
[5]	validation_0-auc:0.713207
[6]	validation_0-auc:0.714284
[7]	validation_0-auc:0.716823
[8]	validation_0-auc:0.717929
[9]	validation_0-auc:0.719771
[10]	validation_0-auc:0.720916
[11]	validation_0-auc:0.722748
[12]	validation_0-auc:0.723486
[13]	validation_0-auc:0.726013
[14]	validation_0-auc:0.72719
[15]	validation_0-auc:0.728113
[16]	validation_0-auc:0.728727
[17]	validation_0-auc:0.729546
[18]	validation_0-auc:0.729859
[19]	validation_0-auc:0.731036
[20]	validation_0-auc:0.731866
[21]	validation_0-auc:0.733299
[22]	validation_0-auc:0.734132
[23]	validation_0-auc:0.734437
[24]	validation_0-auc:0.73561
[25]	validation_0-auc:0.736038
[26]	validation_0-auc:0.73711
[27]	validation_0-auc:0.738165
[28]	validation_0-auc:0.738847
[29]	validation_0-auc:0.739888
[30]	validation_0-auc:0.740717
[31]	validation_0-auc:0.741512
[32]	validation_0-auc

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0.1,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=215, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=11.387150050352467,
       seed=None, silent=None, subsample=0.5, verbosity=1)

In [126]:
predictions = clf.predict(app_test)

In [127]:
submission = pd.DataFrame({'SK_ID_CURR': test_sk_id_curr, 'TARGET': predictions})

In [128]:
submission.head()

,SK_ID_CURR,TARGET
0,100001,0
1,100005,1
2,100013,0
3,100028,0
4,100038,1


In [132]:
submission.to_csv('baseline_xgboost_imputed_median.csv', index = False)